# Gathering and Preprocessing of Data

In this phase , data is collected from various social media sites like Twitter and Reddit.

1. Collecting Data from Reddit

In [1]:
import praw
import pandas as pd
from praw.models import MoreComments
reddit = praw.Reddit(client_id='MjOYmV5QB4Q8bw', client_secret='Ha-Bem-WhpBXOFu6sQExGW1u760', user_agent='Scraping Example')

"""
# get 10 hot posts from the MachineLearning subreddit
hot_posts = reddit.subreddit('Cricket').hot(limit=10)
for post in hot_posts:
    print(post.title)
"""
#comments = []
comments=""
submission = reddit.submission(id="f1zmjl")
submission.comments.replace_more(limit=0)
for top_level_comment in submission.comments:
    comments = comments + top_level_comment.body
    #comments.append([top_level_comment.body])
#comments = pd.DataFrame(comments,columns=['comment'])
print(comments)
#comments.to_pickle('redditdataset.pkl')
comments.to_csv('redditdataset.csv', sep='\t', encoding='utf-8', index=False)

So this is the worst series of Kohli like in years.Saini ODI average at 53 😂SANTNER OVERRATED

NEESHAM OUTDATED

LONG HAVE WE AWAITED

LE BIG MAN HAS BEEN ACTIVATEDKohli running with Shaw is going to be interesting.

Edit: Oh my god![deleted]New bowler : *exists*

India : I am about to make this man's whole career.Fuck i'm excited for the future of long levers Jamieson. A handful to face and his cameo innings was no fluke he can swing the blade. Hit a century belting the piss out of a near full strength English bowling unit in a warm up. [Source](https://www.stuff.co.nz/sport/cricket/102273999/kyle-jamieson-gets-under-englands-skin-on-his-way-to-smashing-surreal-century)LOL Doull saying you need to post it on Twitter if you want Neesham to read itSaini was refusing the single due to his humbleness. He didn't want to show off his skills.NZ chasing down 300 when Taylor and Williamson score less than 40 combined makes me super happy.**Fastest fifty against India in ODIs**

18 Glenn Maxwel

AttributeError: 'str' object has no attribute 'to_csv'

In [2]:
from nltk.tokenize import sent_tokenize
tokenized_text=sent_tokenize(comments)
print(tokenized_text)

['So this is the worst series of Kohli like in years.Saini ODI average at 53 😂SANTNER OVERRATED\n\nNEESHAM OUTDATED\n\nLONG HAVE WE AWAITED\n\nLE BIG MAN HAS BEEN ACTIVATEDKohli running with Shaw is going to be interesting.', 'Edit: Oh my god!', "[deleted]New bowler : *exists*\n\nIndia : I am about to make this man's whole career.Fuck i'm excited for the future of long levers Jamieson.", 'A handful to face and his cameo innings was no fluke he can swing the blade.', 'Hit a century belting the piss out of a near full strength English bowling unit in a warm up.', '[Source](https://www.stuff.co.nz/sport/cricket/102273999/kyle-jamieson-gets-under-englands-skin-on-his-way-to-smashing-surreal-century)LOL Doull saying you need to post it on Twitter if you want Neesham to read itSaini was refusing the single due to his humbleness.', "He didn't want to show off his skills.NZ chasing down 300 when Taylor and Williamson score less than 40 combined makes me super happy.", "**Fastest fifty against 

In [3]:
import nltk
from nltk.tokenize import word_tokenize
tokenized_word=word_tokenize(comments)
print(tokenized_word)

['So', 'this', 'is', 'the', 'worst', 'series', 'of', 'Kohli', 'like', 'in', 'years.Saini', 'ODI', 'average', 'at', '53', '😂SANTNER', 'OVERRATED', 'NEESHAM', 'OUTDATED', 'LONG', 'HAVE', 'WE', 'AWAITED', 'LE', 'BIG', 'MAN', 'HAS', 'BEEN', 'ACTIVATEDKohli', 'running', 'with', 'Shaw', 'is', 'going', 'to', 'be', 'interesting', '.', 'Edit', ':', 'Oh', 'my', 'god', '!', '[', 'deleted', ']', 'New', 'bowler', ':', '*exists*', 'India', ':', 'I', 'am', 'about', 'to', 'make', 'this', 'man', "'s", 'whole', 'career.Fuck', 'i', "'m", 'excited', 'for', 'the', 'future', 'of', 'long', 'levers', 'Jamieson', '.', 'A', 'handful', 'to', 'face', 'and', 'his', 'cameo', 'innings', 'was', 'no', 'fluke', 'he', 'can', 'swing', 'the', 'blade', '.', 'Hit', 'a', 'century', 'belting', 'the', 'piss', 'out', 'of', 'a', 'near', 'full', 'strength', 'English', 'bowling', 'unit', 'in', 'a', 'warm', 'up', '.', '[', 'Source', ']', '(', 'https', ':', '//www.stuff.co.nz/sport/cricket/102273999/kyle-jamieson-gets-under-englands

# Applying TF-IDF to collected dataset

In [4]:
from nltk.probability import FreqDist
fdist = FreqDist(tokenized_word)
print(fdist)

<FreqDist with 2440 samples and 8144 outcomes>


In [5]:
fdist.most_common(2)


[('the', 258), (',', 199)]

In [6]:
# Frequency Distribution Plot
import matplotlib.pyplot as plt
fdist.plot(30,cumulative=False)
plt.show()


<Figure size 640x480 with 1 Axes>

In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))
print(stop_words)

{'up', 's', 'when', 'o', 'ain', 'above', 'll', 'themselves', 'other', "isn't", 'an', 'own', 'your', 'i', 'not', "shouldn't", 'myself', 'those', 'itself', 'from', 'a', 'her', 'yours', 'of', 'didn', 'hadn', 'nor', 'then', 'we', 'they', 'these', 'be', 'into', 'now', 'ma', 'my', "mustn't", 'he', 'have', 'same', 'ours', "won't", 'each', 'me', 'so', 'it', 'them', "you'd", 'off', 'some', 'theirs', 'were', 'shouldn', 'which', "haven't", 'had', 'why', 'd', 'here', 'this', 'under', 'wasn', "hadn't", 'with', 'more', 'whom', 'as', 'for', "you'll", 'once', 'himself', 'needn', 'don', 'should', 'in', 'that', 'how', 'you', 'both', 'our', 'did', 'or', 'until', 'am', 'is', 'do', 'the', "wasn't", 'will', "she's", 'too', 'yourself', 'below', 'mightn', 'all', 'over', 'doing', 'are', 'than', "needn't", 'wouldn', "should've", 've', "shan't", "wouldn't", 'what', 'y', 'aren', 'while', 'but', 'against', "couldn't", "doesn't", 'hasn', 'can', 'such', 'further', 'hers', 'few', 'ourselves', 'where', 'on', 'very', "

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/warlock/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
filtered_sent = []
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)
#print("Tokenized Sentence:",tokenized_text)
print("Filterd Sentence:",filtered_sent)


Filterd Sentence: ['So', 'worst', 'series', 'Kohli', 'like', 'years.Saini', 'ODI', 'average', '53', '😂SANTNER', 'OVERRATED', 'NEESHAM', 'OUTDATED', 'LONG', 'HAVE', 'WE', 'AWAITED', 'LE', 'BIG', 'MAN', 'HAS', 'BEEN', 'ACTIVATEDKohli', 'running', 'Shaw', 'going', 'interesting', '.', 'Edit', ':', 'Oh', 'god', '!', '[', 'deleted', ']', 'New', 'bowler', ':', '*exists*', 'India', ':', 'I', 'make', 'man', "'s", 'whole', 'career.Fuck', "'m", 'excited', 'future', 'long', 'levers', 'Jamieson', '.', 'A', 'handful', 'face', 'cameo', 'innings', 'fluke', 'swing', 'blade', '.', 'Hit', 'century', 'belting', 'piss', 'near', 'full', 'strength', 'English', 'bowling', 'unit', 'warm', '.', '[', 'Source', ']', '(', 'https', ':', '//www.stuff.co.nz/sport/cricket/102273999/kyle-jamieson-gets-under-englands-skin-on-his-way-to-smashing-surreal-century', ')', 'LOL', 'Doull', 'saying', 'need', 'post', 'Twitter', 'want', 'Neesham', 'read', 'itSaini', 'refusing', 'single', 'due', 'humbleness', '.', 'He', "n't", 'wa

In [9]:
filt_sent_string = ' '.join(map(str, filtered_sent))   

import re
only_words = re.findall(r'\w+', filt_sent_string)
print(only_words)
type(only_words)


['So', 'worst', 'series', 'Kohli', 'like', 'years', 'Saini', 'ODI', 'average', '53', 'SANTNER', 'OVERRATED', 'NEESHAM', 'OUTDATED', 'LONG', 'HAVE', 'WE', 'AWAITED', 'LE', 'BIG', 'MAN', 'HAS', 'BEEN', 'ACTIVATEDKohli', 'running', 'Shaw', 'going', 'interesting', 'Edit', 'Oh', 'god', 'deleted', 'New', 'bowler', 'exists', 'India', 'I', 'make', 'man', 's', 'whole', 'career', 'Fuck', 'm', 'excited', 'future', 'long', 'levers', 'Jamieson', 'A', 'handful', 'face', 'cameo', 'innings', 'fluke', 'swing', 'blade', 'Hit', 'century', 'belting', 'piss', 'near', 'full', 'strength', 'English', 'bowling', 'unit', 'warm', 'Source', 'https', 'www', 'stuff', 'co', 'nz', 'sport', 'cricket', '102273999', 'kyle', 'jamieson', 'gets', 'under', 'englands', 'skin', 'on', 'his', 'way', 'to', 'smashing', 'surreal', 'century', 'LOL', 'Doull', 'saying', 'need', 'post', 'Twitter', 'want', 'Neesham', 'read', 'itSaini', 'refusing', 'single', 'due', 'humbleness', 'He', 'n', 't', 'want', 'show', 'skills', 'NZ', 'chasing',

list

In [10]:
mean_words = [i for i in only_words if not i.isdigit()]


final_words = [i for i in mean_words if len(i) > 1]
print(final_words)
type(final_words)

['So', 'worst', 'series', 'Kohli', 'like', 'years', 'Saini', 'ODI', 'average', 'SANTNER', 'OVERRATED', 'NEESHAM', 'OUTDATED', 'LONG', 'HAVE', 'WE', 'AWAITED', 'LE', 'BIG', 'MAN', 'HAS', 'BEEN', 'ACTIVATEDKohli', 'running', 'Shaw', 'going', 'interesting', 'Edit', 'Oh', 'god', 'deleted', 'New', 'bowler', 'exists', 'India', 'make', 'man', 'whole', 'career', 'Fuck', 'excited', 'future', 'long', 'levers', 'Jamieson', 'handful', 'face', 'cameo', 'innings', 'fluke', 'swing', 'blade', 'Hit', 'century', 'belting', 'piss', 'near', 'full', 'strength', 'English', 'bowling', 'unit', 'warm', 'Source', 'https', 'www', 'stuff', 'co', 'nz', 'sport', 'cricket', 'kyle', 'jamieson', 'gets', 'under', 'englands', 'skin', 'on', 'his', 'way', 'to', 'smashing', 'surreal', 'century', 'LOL', 'Doull', 'saying', 'need', 'post', 'Twitter', 'want', 'Neesham', 'read', 'itSaini', 'refusing', 'single', 'due', 'humbleness', 'He', 'want', 'show', 'skills', 'NZ', 'chasing', 'Taylor', 'Williamson', 'score', 'less', 'combin

list

In [11]:
# Converting List to String
filt_sent_string = ' '.join(map(str, final_words))

In [12]:
# Slicing the string into two to perform TF - IDF Algorithm
str_one, str_two = filt_sent_string[:len(filt_sent_string)//2],filt_sent_string[len(filt_sent_string)//2:] 
  
# printing result  
print("The first part of string : " + str_one) 
print("The second part of string : " + str_two) 


The first part of string : So worst series Kohli like years Saini ODI average SANTNER OVERRATED NEESHAM OUTDATED LONG HAVE WE AWAITED LE BIG MAN HAS BEEN ACTIVATEDKohli running Shaw going interesting Edit Oh god deleted New bowler exists India make man whole career Fuck excited future long levers Jamieson handful face cameo innings fluke swing blade Hit century belting piss near full strength English bowling unit warm Source https www stuff co nz sport cricket kyle jamieson gets under englands skin on his way to smashing surreal century LOL Doull saying need post Twitter want Neesham read itSaini refusing single due humbleness He want show skills NZ chasing Taylor Williamson score less combined makes super happy Fastest fifty India ODIs Glenn Maxwell Bengaluru Shahid Afridi Kanpur Dougie Marillier Faridabad de Grandhomme Mount Maunganui Jaddu like gully cricket bro gives strike bat too Saini averages bat ODIs lol Colin need score runs re going dropped team x200B Oh okFor every single o

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [14]:
response = tfidf.fit_transform([str_one, str_two])

In [15]:
feature_names = tfidf.get_feature_names()
for col in response.nonzero()[1]:
    print (feature_names[col], ' - ', response[0, col])

koh  -  0.013042357032680394
definitely  -  0.013042357032680394
origin  -  0.013042357032680394
punjabi  -  0.03912707109804118
khrriya  -  0.013042357032680394
kidhar  -  0.013042357032680394
retire  -  0.013042357032680394
said  -  0.009279744447185989
grandpa  -  0.013042357032680394
row  -  0.009279744447185989
50s  -  0.013042357032680394
afghanistan  -  0.013042357032680394
teams  -  0.013042357032680394
lowest  -  0.013042357032680394
lanka  -  0.009279744447185989
sri  -  0.009279744447185989
put  -  0.009279744447185989
individual  -  0.013042357032680394
looked  -  0.013042357032680394
syllabus  -  0.013042357032680394
championship  -  0.013042357032680394
preparation  -  0.013042357032680394
concentration  -  0.013042357032680394
meis  -  0.013042357032680394
rayuduthakur  -  0.013042357032680394
ambati  -  0.013042357032680394
dube  -  0.009279744447185989
scooby  -  0.013042357032680394
missing  -  0.009279744447185989
spannerpeople  -  0.013042357032680394
toes  -  0.026

praying  -  0.026084714065360788
always  -  0.046398722235929946
53like  -  0.013042357032680394
catch  -  0.009279744447185989
easiest  -  0.013042357032680394
feet  -  0.013042357032680394
twelve  -  0.013042357032680394
potentialjamieson  -  0.013042357032680394
fulfilling  -  0.013042357032680394
finisherkl  -  0.013042357032680394
knock  -  0.013042357032680394
continues  -  0.009279744447185989
form  -  0.018559488894371978
nerdsbrilliant  -  0.013042357032680394
dorky  -  0.013042357032680394
stumps  -  0.03912707109804118
next  -  0.009279744447185989
nerd  -  0.013042357032680394
who  -  0.018559488894371978
was  -  0.013042357032680394
crap  -  0.026084714065360788
espncricinfo  -  0.009279744447185989
zim  -  0.009279744447185989
raina  -  0.009279744447185989
sa  -  0.009279744447185989
eng  -  0.009279744447185989
dhoni  -  0.037118977788743956
year  -  0.009279744447185989
four  -  0.009279744447185989
intentrahul  -  0.013042357032680394
fitness  -  0.013042357032680394


rubberprithvi  -  0.013042357032680394
inspecting  -  0.013042357032680394
ve  -  0.046398722235929946
latham  -  0.027839233341557967
flemingif  -  0.013042357032680394
mccullum  -  0.009279744447185989
brendan  -  0.013042357032680394
wright  -  0.013042357032680394
astle  -  0.013042357032680394
guptill  -  0.027839233341557967
martin  -  0.013042357032680394
zealand  -  0.018559488894371978
opener  -  0.009279744447185989
most  -  0.018559488894371978
die  -  0.013042357032680394
hyped  -  0.013042357032680394
9archer  -  0.013042357032680394
no  -  0.07423795557748791
supreme  -  0.013042357032680394
stay  -  0.013042357032680394
youth  -  0.013042357032680394
free  -  0.027839233341557967
come  -  0.055678466683115935
something  -  0.027839233341557967
fielders  -  0.026084714065360788
runners  -  0.026084714065360788
bad  -  0.046398722235929946
cricketers  -  0.013042357032680394
indian  -  0.09279744447185989
young  -  0.013042357032680394
serieswhy  -  0.013042357032680394
bu

indias  -  0.0
concede  -  0.0
runsouthee  -  0.0
complete  -  0.0
village  -  0.0
pure  -  0.0
lmfao  -  0.0
bed  -  0.0
shitting  -  0.0
overreally  -  0.0
six  -  0.0
mindthakur  -  0.0
wicketsshardul  -  0.0
acceleration  -  0.0
master  -  0.0
selecting  -  0.0
precedent  -  0.0
because  -  0.0
quarters  -  0.0
questions  -  0.0
fail  -  0.0
either  -  0.0
pandyaif  -  0.0
hoonpanty  -  0.0
deta  -  0.0
phirse  -  0.0
jamiesonkl  -  0.0
slog  -  0.0
saving  -  0.0
asset  -  0.0
frame  -  0.0
weakness  -  0.0
baseswe  -  0.0
covers  -  0.0
destructive  -  0.0
incredibly  -  0.0
valuable  -  0.0
gambhir  -  0.0
fewest  -  0.0
zealandamazing  -  0.0
awesome  -  0.0
conditions  -  0.0
booth  -  0.0
grass  -  0.0
huge  -  0.0
picnic  -  0.0
whatnot  -  0.0
lassi  -  0.0
dhokla  -  0.0
faafda  -  0.0
khakra  -  0.0
wt20  -  0.0
till  -  0.0
able  -  0.0
endless  -  0.0
patch  -  0.0
purple  -  0.0
tape  -  0.0
luck  -  0.0
bois  -  0.0
finals  -  0.0
tournament  -  0.0
272i  -  0.0
imagi

1st  -  0.0
klpd  -  0.0
quality  -  0.0
true  -  0.0
mitch  -  0.0
itfucking  -  0.0
liked  -  0.0
stadium  -  0.0
chinaswamy  -  0.0
huss  -  0.0
coffee  -  0.0
corneronce  -  0.0
collapse  -  0.0
el  -  0.0
known  -  0.0
legend  -  0.0
continue  -  0.0
reset  -  0.0
sanjuiyer  -  0.0
bias  -  0.0
commentators  -  0.0
thingy  -  0.0
tired  -  0.0
fatigue  -  0.0
120s  -  0.0
130s  -  0.0
mid  -  0.0
low  -  0.0
considering  -  0.0
bowlingneesham  -  0.0
reality  -  0.0
christ  -  0.0
coming  -  0.0
praise  -  0.0
hating  -  0.0
haydenwtf  -  0.0
peachvirat  -  0.0
seriesoh  -  0.0
baby  -  0.0
practice  -  0.0
including  -  0.0
spanking  -  0.0
thank  -  0.0
standard  -  0.0
spare  -  0.0
rr  -  0.0
needing  -  0.0
thrown  -  0.0
nicholls  -  0.0
shocking  -  0.0
sisterfrom  -  0.0
wife  -  0.0
mum  -  0.0
yer  -  0.0
in  -  0.0
mellon  -  0.0
york  -  0.0
of  -  0.0
thicc  -  0.0
daddy  -  0.0
papi  -  0.0
sink  -  0.0
kitchen  -  0.0
throws  -  0.0
prisoners  -  0.0
takes  -  0.0
v

played  -  0.046398722235929946
best  -  0.046398722235929946
let  -  0.037118977788743956
batsman  -  0.037118977788743956
got  -  0.046398722235929946
option  -  0.018559488894371978
pressure  -  0.009279744447185989
order  -  0.10207718891904588
middle  -  0.0835177000246739
opportunity  -  0.009279744447185989
half  -  0.009279744447185989
everyone  -  0.018559488894371978
plays  -  0.009279744447185989
swear  -  0.009279744447185989
pant  -  0.055678466683115935
kid  -  0.046398722235929946
ross  -  0.009279744447185989
ve  -  0.046398722235929946
latham  -  0.027839233341557967
mccullum  -  0.009279744447185989
guptill  -  0.027839233341557967
zealand  -  0.018559488894371978
opener  -  0.009279744447185989
most  -  0.018559488894371978
no  -  0.07423795557748791
free  -  0.027839233341557967
come  -  0.055678466683115935
something  -  0.027839233341557967
bad  -  0.046398722235929946
indian  -  0.09279744447185989
bumrah  -  0.12991642226060385
it  -  0.018559488894371978
captai

In [ ]:
from nltk.probability import FreqDist
fdist = FreqDist(final_words)
print(fdist)

In [ ]:
print("Fans mostly discussed about :")
fdistlist = fdist.most_common(10)
print(fdistlist)

In [ ]:
fdiststr = ' '.join(map(str, fdistlist))
tokenized_list = word_tokenize(fdiststr)
tokenized_list = ' '.join(map(str, tokenized_list))
key_comments = re.findall(r'\w+', tokenized_list)
key_comments = [i for i in key_comments if not i.isdigit()]
#print(key_comments)

In [ ]:
import matplotlib.pyplot as plt
fdist.plot(30,cumulative=False)
plt.show()

In [ ]:
type(key_comments)
print(key_comments)

In [ ]:
import csv

with open('redditdataset.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

print(data)

In [ ]:
trial = []
for i in key_comments:
    if i in data:
        trial.append(i)
print(trial)

In [ ]:
res = [idx for idx in test_list if idx[0].lower() == check.lower()]